In [ ]:
%%writefile main.cu
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <chrono>
// TODO: Add header files as needed
#include <cuda_runtime.h>

#define NUM_PARTICLES 100
#define NUM_FRAMES 500
#define DIMENSIONS 3

  // TODO: Your helper function goes here

  // TODO: End of your helper function

float random_float(float min, float max) {
   float scale = rand() / (float) RAND_MAX;
   return min + scale * (max - min);
}

int main(int argc, char *argv[]) {
   srand(time(NULL));
   float (*particles_position)[DIMENSIONS] = (float (*)[DIMENSIONS])malloc(NUM_PARTICLES * NUM_FRAMES * sizeof(float[DIMENSIONS]));
   float (*particles_speed)[DIMENSIONS] = (float (*)[DIMENSIONS])malloc(NUM_PARTICLES * NUM_FRAMES * sizeof(float[DIMENSIONS]));
   float gravity = 0.98f;
   float time_step = 0.05f;

   for (int particle = 0; particle < NUM_PARTICLES; particle++)
   {
      particles_position[particle][0] = random_float(-0.01f, 0.01f); // x
      particles_position[particle][1] = random_float(-0.01f, 0.01f); // y
      particles_position[particle][2] = random_float(0.01f, 0.01f); // z
      particles_speed[particle][0] = random_float(-0.1f, 0.1f); // v_x
      particles_speed[particle][1] = random_float(-0.1f, 0.1f); // v_y
      particles_speed[particle][2] = random_float(0.5f, 1.0f); // v_z
   }

   cudaDeviceSynchronize();
   auto start = std::chrono::high_resolution_clock::now();

   // TODO: Your implementation goes here

   // TODO: End of your implementation

   cudaDeviceSynchronize();
   auto end = std::chrono::high_resolution_clock::now();
   auto duration = std::chrono::duration_cast<std::chrono::milliseconds>(end - start);
   printf("Time taken: %ld milliseconds\n", duration.count());

   FILE *file = fopen("particle_data.bin", "wb");
   if (file == NULL) {
      fprintf(stderr, "Failed to open file for writing\n");
      free(particles_position);
      free(particles_speed);
      return 1;
   }

   size_t written = fwrite(particles_position, sizeof(float), NUM_PARTICLES * NUM_FRAMES * DIMENSIONS, file);
   if (written != NUM_PARTICLES * NUM_FRAMES * DIMENSIONS) {
      fprintf(stderr, "Failed to write all data to file\n");
   } else {
      printf("Successfully wrote %zu floats to particle_data.bin\n", written);
      printf("Matrix dimensions: %d particles × %d frames × %d coordinates\n",
             NUM_PARTICLES, NUM_FRAMES, DIMENSIONS);
   }

   fclose(file);
   free(particles_position);
   free(particles_speed);
   return 0;
}

In [ ]:
!nvcc -arch=sm_70 main.cu -o main.ex

In [ ]:
!./main.ex

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D
import os
import io
from IPython.display import HTML, display

# Constants
NUM_PARTICLES = 100
NUM_FRAMES = 500
DIMENSIONS = 3

def read_binary_data(filename):
   """Read the binary data file containing particle positions."""
   if not os.path.exists(filename):
      raise FileNotFoundError(f"Binary data file {filename} not found")
   data = np.fromfile(filename, dtype=np.float32)
   data = data.reshape(NUM_FRAMES, NUM_PARTICLES, DIMENSIONS)

   return data

def create_animation(data, save_gif=True, output_file="particle_animation.gif"):
   """Create and save an animation of the particle movement."""
   fig = plt.figure(figsize=(10, 8))
   ax = fig.add_subplot(111, projection='3d')
   max_val = np.max(np.abs(data))
   ax.set_xlim([-max_val, max_val])
   ax.set_ylim([-max_val, max_val])
   ax.set_zlim([-max_val, max_val])
   ax.set_xlabel('X')
   ax.set_ylabel('Y')
   ax.set_zlabel('Z')
   ax.set_title('Particle Animation')
   colors = plt.cm.viridis(np.linspace(0, 1, NUM_PARTICLES))
   scatter = ax.scatter(data[0, :, 0], data[0, :, 1], data[0, :, 2],
                      c=colors, s=30, alpha=0.8)

   def update(frame):
      scatter._offsets3d = (data[frame, :, 0],
                          data[frame, :, 1],
                          data[frame, :, 2])
      ax.set_title(f'Particle Animation - Frame {frame+1}/{NUM_FRAMES}')
      return scatter,

   ani = FuncAnimation(fig, update, frames=NUM_FRAMES,
                      interval=42, blit=False)
   if save_gif:
      ani.save(output_file, writer='pillow', fps=10)
      print(f"Animation saved as {output_file}")
   plt.close()
   return ani

def display_animation_in_colab(animation):
   """Display animation in Colab using HTML."""
   try:
      from IPython.display import HTML
      from matplotlib.animation import HTMLWriter
      import matplotlib as mpl

      import google.colab
      is_colab = True
   except:
      is_colab = False

   if is_colab:
      print("Displaying animation using HTML in Colab...")
      return HTML(animation.to_jshtml())
   else:
      print("Not in Colab environment. Showing animation using plt.show()")
      plt.figure()
      return animation

if __name__ == "__main__":
   data_file = "particle_data.bin"

   try:
      particle_data = read_binary_data(data_file)
      animation = create_animation(particle_data)
      html_animation = display_animation_in_colab(animation)
      display(html_animation)
   except Exception as e:
      print(f"Error: {e}")
      import traceback
      traceback.print_exc()